
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# File Explosion
We see many data engineers partitioning their tables in ways that can cause major performance issues, without improving future query performance. This is called "over partitioning". We'll see what that looks like in practice in this demo.

##### Useful References
- [Partitioning Recommendations](https://docs.databricks.com/en/tables/partitions.html)
- [CREATE TABLE Syntax](https://docs.databricks.com/en/sql/language-manual/sql-ref-syntax-ddl-create-table-using.html)
- [About ZORDER](https://docs.databricks.com/en/delta/data-skipping.html)
- [About Liquid Clustering](https://docs.databricks.com/en/delta/clustering.html)

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default. If you use Serverless, errors will be returned when setting compute runtime properties.

Follow these steps to select the classic compute cluster:

1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

    - In the drop-down, select **More**.

    - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

1. Find the triangle icon to the right of your compute cluster name and click it.

1. Wait a few minutes for the cluster to start.

1. Once the cluster is running, complete the steps above to select your cluster.

## A. Classroom Setup

Run the following cell to configure your working environment for this course. It will also set your default catalog to your unique **labuser** catalog, and the default schema to **default**. All tables will be read from and written to this location.
<br></br>

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course.

In [0]:
%run ./Includes/Classroom-Setup-1

Let's Check the Current Catalog and Schema

In [0]:
%sql
SELECT current_catalog(), current_schema()

## B. Disable Caching

Run the following cell to set a Spark configuration variable that disables disk caching.

Turning disk caching off prevents Databricks from storing cloud storage files after the first query. This makes the effect of the optimizations more apparent by ensuring that files are always pulled from cloud storage for each query.

For more information, see [Optimize performance with caching on Databricks](https://docs.databricks.com/en/optimizations/disk-cache.html#optimize-performance-with-caching-on-databricks).

**NOTE:** This will not work in Serverless. Please use classic compute to turn off caching. If you're using Serverless, an error will be returned.

In [0]:
spark.conf.set('spark.databricks.io.cache.enabled', False)


## C. Process & Write IoT data
Let's generate some fake IoT data. This first time around, we are only going to generate 2,500 rows.

In [0]:
from pyspark.sql.functions import *

df = (spark
      .range(0, 2500)
      .select(
          hash('id').alias('id'), # randomize our ids a bit
          rand().alias('value'),
          from_unixtime(lit(1701692381 + col('id'))).alias('time') 
      ))

df.display()

Now we'll write the data to a table partitioned by **id** (2,500 distinct values), which will result in every row being written to a separate folder for each partition. Writing 2,500 rows in this fashion will take a long time, as we are creating 2,500 partitions. Each partition will contain a folder with one file, and each file will store one row of data for each **id**, leading to the 'small file' problem.

Note how long it takes to generate the table.

**NOTE:** This will take about 1-2 minutes to create the table with 2,500 partitions.


In [0]:
spark.sql('DROP TABLE IF EXISTS iot_data_partitioned')

(df
 .write
 .mode('overwrite')
 .option("overwriteSchema", "true")
 .partitionBy('id')
 .saveAsTable("iot_data_partitioned")
)

Describe the history of the **iot_data_partitioned** table. Confirm the following:
- In the **operationParameters** column, the table is partitioned by **id**.
- In the **operationMetrics** column, the table contains 2,500 files, one parquet file for each unique partitioned **id**.

In [0]:
%sql
DESCRIBE HISTORY iot_data_partitioned;

You can use the `SHOW PARTITIONS` statement to list all partitions of a table. Run the code and view the results. Notice that the table is partitioned by **id** and contains 2,500 rows.


In [0]:
%sql
SHOW PARTITIONS iot_data_partitioned;

### C1. Query the Table
Run the two queries against the partitioned table we just created. Note the time taken to execute each query.


#### Query 1 - Filter by the partitioned id column
**NOTE:** (1-2 second execution)

In [0]:
%sql
-- Query 1: Filtering by the partitioned column. 
SELECT * 
FROM iot_data_partitioned 
WHERE id = 519220707;

Let's see how this query performed using the Spark UI. Note in particular the amount of cloud storage requests and their associated time. To view how the query performed complete the following:

1. In the cell above, expand **Spark Jobs**.

2. Right click on **View** and select *Open in a New Tab*. 

    **NOTE:** In the Vocareum lab environment if you click **View** without opening it in a new tab the pop up window will display an error.

3. In the new window, find the **SQL/DataFrame Properties** header at the top and select the number.

4. Here you should see the entire query plan.

5. In the query plan, scroll down to the bottom and find **PhotonScan parquet dbacademy.*your schema*.iot_data_partitioned (1)** and select the plus icon.


![1.1-query-1-iot_partitioned.png](./Includes/images/1.1-query-1-iot_partitioned.png)

#### Look at the following metrics in the Spark UI:


| Metric    | Value    | Note    |
|-------------|-------------|-------------|
| cloud storage request count| 1| Refers to the number of requests made to the cloud storage systems like S3, Azure Blob, or Google Cloud Storage during job execution. This could involve multiple operations like reading metadata, accessing directories, or fetching the actual data. <br></br>Monitoring this metric helps optimize performance, reduce costs, and identify potential inefficiencies in data access patterns. |
| cloud storage response size| 880.0B|  Indicates the total amount of data transferred from cloud storage to Spark during the execution of a job. It helps track the volume of data read or written to cloud storage, providing insights into I/O performance and potential bottlenecks related to data transfer.|
| files pruned | 2,499 |Indicates the number of files that Spark skipped or ignored during a job execution. A total of 2,499 files were skipped by Spark due to pruning based on the query filtering by **id**. This is due to the table being partitioned by **id**, the queried column. Spark reads only the necessary partitions for processing and skips the other partitions.|
| files read | 1 |  Indicates the number of files that Spark has actually read during job execution. Here, 1 file was read during the execution of the Spark job. Only 1 file was read because the query was executed on the partitioned **id** column. Spark only needs to read the necessary partitions(s) based on the query.

#### Summary
Because the data was partitioned by **id** and queried by the partitioned column, Spark will only read the necessary partition(s) (one partition in this example) and will skip the other partitioned files.

#### Query 2 - Filter by a column that is not partitioned 
**NOTE:** (5-10 second execution)

In [0]:
%sql
SELECT avg(value) 
FROM iot_data_partitioned 
WHERE time >= "2023-12-04 12:19:00" AND
      time <= "2023-12-04 13:01:20";

Let's see how this query performed using the Spark UI. Note in particular the amount of cloud storage requests and their associated time. To view how the query performed complete the following:

1. In the cell above, expand **Spark Jobs**.

2. Right click on **View** and select *Open in a New Tab*. 

    **NOTE:** In the Vocareum lab environment if you click **View** without opening it in a new tab the pop up window will display an error.

3. In the new window, find the **SQL/DataFrame Properties** header at the top and select the number.

4. Here you should see the entire query plan.

5. In the query plan, scroll down to the bottom and find **PhotonScan parquet dbacademy.*your schema*.iot_data_partitioned (1)** and select the plus icon.

#### Look at the following metrics in the Spark UI (results may vary):
| Metric    | Value    | Note    |
|-------------|-------------|-------------|
| cloud storage request count total (min, med, max)| 2500 (21, 37, 37)| Refers to the number of requests made to the cloud storage systems like S3, Azure Blob, or Google Cloud Storage during job execution. This could involve multiple operations like reading metadata, accessing directories, or fetching the actual data. <br></br>Monitoring this metric helps optimize performance, reduce costs, and identify potential inefficiencies in data access patterns. <br></br>The min, med and max represent the summary of requests made by tasks or executors. The distribution is fairly uniform across tasks or executors and there is not a large variance in the number of cloud storage requests made by each task.|
| cloud storage response size total (min, med, max)| 2.1 MiB (18.0 KiB, 31.8 KiB, 31.8 KiB)| Indicates the total amount of data transferred from cloud storage to Spark during the execution of a job. It helps track the volume of data read or written to cloud storage, providing insights into I/O performance and potential bottlenecks related to data transfer.<br></br> The min,med and max indicate most tasks transferring between 18.0 KiB and 31.8 KiB of data, showing a relatively consistent and uniform data transfer pattern across tasks.|
| files pruned | 0 |A total of 0 files were skipped by Spark due to pruning based on the query's filters. This is due to the table being partitioned by **id** but queried by the **time** column. No files were pruned in this query.|
| files read | 2,500 | 2,500 files were read during the execution of the Spark job. This is because of the query was executed on the **time** column and the table is partitioned **id** column. In this query, all files were read into Spark then filtered for the necessary rows.|

#### Summary
Because the data was partitioned by **id** but queried by the **time** column, Spark read all of the files to perform the required query and filter the data to return a single row.

## D. Fixing the Problem

Up to this point, we have been working with 2,500 rows of data that were partitioned in the table.

We are now going to increase the volume dramatically by using 50,000,000 rows. If we had tried the code above with a dataset this large, it would have taken considerably longer to create all of the partitions (directories for each partition).

As before, the following cell generates the data.


In [0]:
from pyspark.sql.functions import *

df = (spark
      .range(0,50000000, 1, 32)
      .select(
          hash('id').alias('id'), # randomize our ids a bit
          rand().alias('value'),
          from_unixtime(lit(1701692381 + col('id'))).alias('time') 
      )
    )

df.display()


Now we'll create a table named **iot_data** to capture the data, **this time without partitioning**. Doing it this way accomplishes the following:
- Takes less time to run, even on larger datasets, because we are not creating a high number of table partitions.
- Writes fewer files (32 files for 50,000,000 rows vs. 2,500 files for 2,500 rows in the partitioned table).
- Writes faster compared to disk partitioning because all files are in one directory instead of creating 2,500 directories.
- Queries for one **id** in about the same time as before.
- Filters by the **time** column much faster since it only has to query one directory.


In [0]:
spark.sql('DROP TABLE IF EXISTS iot_data')

(df
 .write
 .option("overwriteSchema", "true")
 .mode('overwrite')
 .saveAsTable("iot_data")
)

display(spark.sql('SELECT count(*) FROM iot_data'))

Describe the history of the **iot_data** table. Confirm the following:
- In the **operationParameters** column, confirm the table is not partitioned.
- In the **operationMetrics** column, confirm the table contains 32 total files in the table.

In [0]:
%sql
DESCRIBE HISTORY iot_data;

## E. Validate Optimization
The next two cells repeat the queries from earlier and will put this change to the test. The first cell should run almost as fast as before, and the second cell should run much faster.

### E1. Query 1 - Filter by the id column (non partitioned table)

In [0]:
%sql
SELECT * 
FROM iot_data 
WHERE id = 519220707

Let's see how this query performed using the Spark UI. Compare the results against the same query we performed earlier against an over-partitioned table.

1. In the cell above, expand **Spark Jobs**.

2. Right click on **View** and select *Open in a New Tab*. 

    **NOTE:** In the Vocareum lab environment if you click **View** without opening it in a new tab the pop up window will display an error.

3. In the new window, find the **SQL/DataFrame Properties** header at the top and select the number.

4. Here you should see the entire query plan.

5. In the query plan, scroll down to the bottom and find **PhotonScan parquet dbacademy.*your schema*.iot_data (1)** and select the plus icon.

#### Look at the following metrics in the Spark UI (results may vary):
| Metric    | Value    | Note    |
|-------------|-------------|-------------|
| cloud storage request count total (min, med, max)| 65 (8, 8, 9)|  Refers to the number of requests made to the cloud storage systems like S3, Azure Blob, or Google Cloud Storage during job execution. This could involve multiple operations like reading metadata, accessing directories, or fetching the actual data. <br></br>Monitoring this metric helps optimize performance, reduce costs, and identify potential inefficiencies in data access patterns.<br></br>The request count distribution is quite uniform across tasks/executors, as the min, med, and max values are very close to each other (8 and 9) indicating cloud storage access was consistent during execution.|
| cloud storage response size total (min, med, max)| 216.9 MiB (24.8 MiB, 24.8 MiB, 43.0 MiB)| Indicates the total amount of data transferred from cloud storage to Spark during the execution of a job. It helps track the volume of data read or written to cloud storage, providing insights into I/O performance and potential bottlenecks related to data transfer.|
| files pruned | 0 |A total of 0 files were skipped by Spark due to pruning based on the query's filters. This is because no optimized saving table techniques were used for the table.|
| files read | 32| 32 files were read during the execution of the Spark job. |

#### Summary
In this example, we had 50,000,000 rows (more than the original 2,500 rows) but only 32 files and no partition in the table. While this table had many more rows, Spark only had 32 files and no partitions to query, avoiding the small file problem we encountered in the partitioned table, enabling the query to run fast.

### E2. Query 2 - Filter by the time column (non partitioned table)

In [0]:
%sql
SELECT avg(value) 
FROM iot_data 
WHERE time >= "2023-12-04 12:19:00" AND 
      time <= "2023-12-04 13:01:20";

Let's see how this query performed using the Spark UI. Compare the results against the same query we performed earlier against an over-partitioned table.

1. In the cell above, expand **Spark Jobs**.

2. Right click on **View** and select *Open in a New Tab*. 

    **NOTE:** In the Vocareum lab environment if you click **View** without opening it in a new tab the pop up window will display an error.

3. In the new window, find the **SQL/DataFrame Properties** header at the top and select the number.

4. Here you should see the entire query plan.

5. In the query plan, scroll down to the bottom and find **PhotonScan parquet dbacademy.*your schema*.iot_data (1)** and select the plus icon.

#### Look at the following metrics in the Spark UI (results may vary):

| Metric    | Value    | Note    |
|-------------|-------------|-------------|
| cloud storage request count| 3| Refers to the number of requests made to the cloud storage systems like S3, Azure Blob, or Google Cloud Storage during job execution. This could involve multiple operations like reading metadata, accessing directories, or fetching the actual data. |
| cloud storage response size| 	18.4 MiB| Indicates the total amount of data transferred from cloud storage to Spark during the execution of a job. It helps track the volume of data read or written to cloud storage, providing insights into I/O performance and potential bottlenecks related to data transfer.|
| files pruned | 31 | Spark determined  that 31 files did not contain any relevant data based on the WHERE condition filter for the **time** column. |
| files read | 1 | Spark only read 1 of the files from cloud storage. |

#### Summary
In this example, we had 50,000,000 (more than the original 2,500 rows) but only 32 files in the table. While this table had many more rows, Spark only had 32 files to query, pruning almost all the files based on the **time** column, avoiding the small file problem we encountered in the partitioned table.

### Demo Summary
In the **iot_data** table, we did not partition the table when saving it. In this example, we allowed Spark to handle the saving process. Even though the dataset was much larger than the partitioned table from the first example, Spark optimized how the data was saved. It created 32 files for the Delta table, with each file containing a balanced number of rows, thus avoiding the "small file" problem that occurred with the partitioned table in the earlier example.

&copy; 2026 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>